<font color="lightblue">

**GERMON Paul et MARTIN Hugo**

</font>

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization
import tensorflow.keras.optimizers as optimizers

In [3]:
with open('pickle_enlarged_X', 'rb') as fx:
    enlarged_X = pickle.load(fx)

with open('pickle_enlarged_Y', 'rb') as fy:
    enlarged_Y = pickle.load(fy)
    
print("enlarged_X.shape = ", enlarged_X.shape)
print("enlarged_Y.shape = ", enlarged_Y.shape)

enlarged_X.shape =  (331584, 15, 9, 9)
enlarged_Y.shape =  (331584,)


In [4]:
# Séparation des données d'entrée en un training set et un test set
X_train, X_test, Y_train, Y_test = train_test_split(enlarged_X, enlarged_Y, test_size=0.33, shuffle=True)

In [5]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"Y_train.shape = {Y_train.shape}")
print(f"Y_test.shape = {Y_test.shape}")

X_train.shape = (222161, 15, 9, 9)
X_test.shape = (109423, 15, 9, 9)
Y_train.shape = (222161,)
Y_test.shape = (109423,)


# Model training

In [7]:
model_priors = Sequential()

# 1 Conv2D 128 filters
model_priors.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

# 3 Conv2D 64 filters
model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

# 5 Conv2D 32 filters
model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))


# 3 Conv 2D 16 filters
model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Conv2D(filters = 16,
                 kernel_size = (3,3),
                 padding = 'same',
                 data_format = 'channels_first',
                 input_shape = X_train.shape[1:]))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))

model_priors.add(Flatten())

# 3 Dense 160 neurons
model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(160))
model_priors.add(BatchNormalization())
model_priors.add(Activation('relu'))
model_priors.add(Dropout(0.5))

model_priors.add(Dense(1, activation='relu'))

model_priors.compile(loss='mse',
              optimizer='adam',
              metrics=['mae', 'mse'])

In [8]:
model_priors.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 9, 9)         17408     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 9, 9)         36        
_________________________________________________________________
activation (Activation)      (None, 128, 9, 9)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 9, 9)          73792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 9, 9)          36        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 9, 9)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 9, 9)         

In [9]:
epochs = 30
batch_size = 128

history = model_priors.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),
          shuffle=True)

Epoch 1/30
1736/1736 [==============================] - 25s 14ms/step - loss: 0.1425 - mae: 0.2944 - mse: 0.1425 - val_loss: 0.1126 - val_mae: 0.2762 - val_mse: 0.1126
Epoch 2/30
1736/1736 [==============================] - 24s 14ms/step - loss: 0.0764 - mae: 0.2062 - mse: 0.0764 - val_loss: 0.0586 - val_mae: 0.1878 - val_mse: 0.0586
Epoch 3/30
1736/1736 [==============================] - 24s 14ms/step - loss: 0.0502 - mae: 0.1574 - mse: 0.0502 - val_loss: 0.1123 - val_mae: 0.2331 - val_mse: 0.1123
Epoch 4/30
1736/1736 [==============================] - 25s 14ms/step - loss: 0.0428 - mae: 0.1418 - mse: 0.0428 - val_loss: 0.0411 - val_mae: 0.1316 - val_mse: 0.0411
Epoch 5/30
1736/1736 [==============================] - 24s 14ms/step - loss: 0.0394 - mae: 0.1344 - mse: 0.0394 - val_loss: 0.0434 - val_mae: 0.1327 - val_mse: 0.0434
Epoch 6/30
1736/1736 [==============================] - 24s 14ms/step - loss: 0.0367 - mae: 0.1280 - mse: 0.0367 - val_loss: 0.0381 - val_mae: 0.1214 - val_mse:

In [10]:
score = model_priors.evaluate(X_test, Y_test, verbose=0)

In [11]:
print(len(score))
print('Test loss:', score[0])
print('Test mae:', score[1])
print('Test mse:', score[2])

3
Test loss: 0.030031021684408188
Test mae: 0.09688544273376465
Test mse: 0.030031021684408188


In [12]:
path = "C:/Users/Lenovo/Documents/Paul/Travail/ENSEIRB-MATMECA/3A/Alpha-Go/alpha-go-like/priors/"
model_priors.save('model_priors')

model_priors_json = model_priors.to_json()
with open("model_priors.json", "w") as json_file:
    json_file.write(model_priors_json)
# serialize weights to HDF5
model_priors.save_weights("model_priors.h5")
print("Saved model to disk")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_priors/assets
Saved model to disk


In [13]:
X_10 = X_test[:10]
Y_10 = Y_test[:10]
print(X_10.shape)
print(Y_10.shape)

Y_pred_10 = model_priors.predict(X_10)

for i in range(10):
  print(f"Sample {i}: predicted = {Y_pred_10[i]} / true = {Y_10[i]}")

(10, 15, 9, 9)
(10,)
Sample 0: predicted = [0.] / true = 0.0
Sample 1: predicted = [0.] / true = 0.0
Sample 2: predicted = [0.] / true = 0.03
Sample 3: predicted = [0.38198596] / true = 0.3
Sample 4: predicted = [1.0056674] / true = 1.0
Sample 5: predicted = [0.37277603] / true = 0.24
Sample 6: predicted = [0.5180142] / true = 0.32
Sample 7: predicted = [0.6262366] / true = 0.62
Sample 8: predicted = [0.6480617] / true = 0.76
Sample 9: predicted = [0.24482161] / true = 0.13
